##### Pacotes Necessários 

In [ ]:
import psycopg2
import pandas as pd
from configparser import ConfigParser

##### Conexão com a Base e Extração de Dados

In [ ]:
# Função para configuração de conexão com parâmetros de acesso presentes no arquivo database.ini
# Transforma o arquivo em uma variável de objeto
def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Seção {0} não encontrada no arquivo {1}'.format(section, filename))
    return db

In [ ]:
def connect():
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()

        retorno = []

        cursor.execute('SELECT * FROM forms')
        forms = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'forms'")
        colunas = cursor.fetchall()
        retorno.append([forms, colunas])

        cursor.execute('SELECT * FROM orders')
        orders = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders'")
        colunas = cursor.fetchall()
        retorno.append([orders, colunas])

        cursor.execute('SELECT * FROM order_items')
        items = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'order_items'")
        colunas = cursor.fetchall()
        retorno.append([items, colunas])

        cursor.close()
        return retorno
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Conexão encerrada.')


if __name__ == '__main__':
    retorno = connect()

In [ ]:
forms = pd.DataFrame(retorno[0][0])
forms.columns = [item for sublist in retorno[0][1] for item in sublist]

orders = pd.DataFrame(retorno[1][0])
orders.columns = [item for sublist in retorno[1][1] for item in sublist]

order_items = pd.DataFrame(retorno[2][0])
order_items.columns = [item for sublist in retorno[2][1] for item in sublist]

##### Manipulação de dados

In [ ]:
# Remoção de colunas
forms = forms.drop(['id_cliente','data','sistema_operacional','navegador','localizacao','locale','cidade','utm_source',
'caracteristica','tempo_procedimento'], axis = 1)

orders = orders[['id_pedido','target']]

order_items = order_items[['id_pedido','id_form']]

In [ ]:
# Selecionando apenas os valores únicos da coluna 'id_form' da tabela 'order_items'
items_form = order_items.drop_duplicates(subset = ['id_form'])
# Left join da tabela 'forms' com a 'items_form' seguido de left join da tabela resultante com a tabela 'orders'. 
# df é a base de dados resultante
df = forms.merge(items_form, 
                 on = 'id_form', 
                 how = 'left').merge(orders, 
                                     on = 'id_pedido', 
                                     how = 'left').drop(['id_form','id_pedido'], axis = 1)

In [ ]:
# Lidando com NAs
df.target.fillna(0, inplace = True)
df.dropna(axis = 0, inplace = True)
df.drop(df[df.efeitos_desejados == 'N/C'].index, axis = 0, inplace = True)
df.drop(df[df.procedimentos == 'N/C'].index, axis = 0, inplace = True)


In [ ]:
# # Transformação de variáveis
# [df.estado, codes_estado] = pd.factorize(df.estado)
# [df.tipo_cabelo, codes_tipo_cabelo] = pd.factorize(df.tipo_cabelo)
# [df.comprimento, codes_comprimento] = pd.factorize(df.comprimento)
# [df.tipo_fios, codes_tipo_fios] = pd.factorize(df.tipo_fios)
# [df.dieta, codes_dieta] = pd.factorize(df.dieta)
# [df.atividade_fisica, codes_atividade_fisica] = pd.factorize(df.atividade_fisica)
# [df.frequencia_estresse, codes_frequencia_estresse] = pd.factorize(df.frequencia_estresse)
# [df.faixa_etaria, codes_faixa_etaria] = pd.factorize(df.faixa_etaria)
# [df.fragancia, codes_fragancia] = pd.factorize(df.fragancia)

In [ ]:
df['efeito1'] = 0
df['efeito2'] = 0
df['efeito3'] = 0
df['efeito4'] = 0
df['efeito5'] = 0
df['efeito6'] = 0
df['efeito7'] = 0
df['efeito8'] = 0
df['efeito10'] = 0

df.loc[df['efeitos_desejados'].str.contains('Efeito 1'), 'efeito1'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 2'), 'efeito2'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 3'), 'efeito3'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 4'), 'efeito4'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 5'), 'efeito5'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 6'), 'efeito6'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 7'), 'efeito7'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 8'), 'efeito8'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 10'), 'efeito10'] = 1

In [ ]:
df['procedimentos1'] = 0
df['procedimentos2'] = 0
df['procedimentos3'] = 0
df['procedimentos4'] = 0
df['procedimentos5'] = 0

df.loc[df['procedimentos'].str.contains('Nenhum'), 'procedimentos1'] = 1
df.loc[df['procedimentos'].str.contains('Tintura'), 'procedimentos2'] = 1
df.loc[df['procedimentos'].str.contains('Corte agressivo'), 'procedimentos3'] = 1
df.loc[df['procedimentos'].str.contains('Exposição ao sol'), 'procedimentos4'] = 1
df.loc[df['procedimentos'].str.contains('Nenhum'), 'procedimentos5'] = 1

In [ ]:
df.drop(['efeitos_desejados', 'procedimentos'], axis = 1, inplace=True)

In [ ]:
coluna = df.pop('target')
df = df.assign(target = coluna).astype('int64')

In [ ]:
df.info()